In [1]:
import sys, platform
print("PY:", sys.executable)
print("VER:", sys.version)
print("PLATFORM:", platform.platform())


PY: /Users/jan/Documents/working papers/project 1/.moirai/bin/python
VER: 3.13.6 (v3.13.6:4e665351082, Aug  6 2025, 11:22:35) [Clang 16.0.0 (clang-1600.0.26.6)]
PLATFORM: macOS-15.6.1-arm64-arm-64bit-Mach-O


In [ ]:
# --- Optional installs (run once in your venv) ---
# pip install -U numpy pandas matplotlib
# pip install -U torch --extra-index-url https://download.pytorch.org/whl/cpu  
# pip install uni2ts gluonts huggingface_hub
#
# On macOS Apple Silicon for other DL libs (not needed here):
# pip install tensorflow tensorflow-macos tensorflow-metal  # if you use TF elsewhere
# pip install jax jax-metal  # if you use JAX elsewhere

# source .moirai311/bin/activate  


In [9]:
from pathlib import Path
import os
import pandas as pd

# Project root (adjust if needed)
project_root = Path("/Users/jan/Documents/working papers/project 1")
data_dir     = project_root / "data"
out_dir      = project_root / "models"
out_dir.mkdir(parents=True, exist_ok=True)

# Input price CSV (must have a datetime index or a 'timestamp' column)
# Example path; point to your actual BTC file:
IN_CSV       = data_dir / "btc_1h_test.csv"

# Output predictions
OUT_PRED_CSV = out_dir / "Moirai_zeroshot_btc_preds.csv"

# Core forecast config (TTM-style)
CONTEXT_LENGTH    = 512
PREDICTION_LENGTH = 1
FREQ              = "1H"     # 15-minute data ('1H' for hourly, etc.)

# Moirai adapter config (filled via env or kwargs)
# You can either set these here or export them before launching Jupyter.
os.environ.setdefault("MOIRAI_REPO_OR_DIR", "/absolute/path/to/moirai/checkpoint_or_repo")
os.environ.setdefault("MOIRAI_CLASS",      "my_moirai_pkg.core:MoiraiModule")  # example; replace with your class
PATCH_SIZE   = 16
ROPE_SCALE   = 1.0

In [10]:
import numpy as np

# Load CSV; expecting 'timestamp' + 'price' columns or similar.
df = pd.read_csv(IN_CSV)

# Normalize to a DatetimeIndex
if "timestamp" in df.columns:
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=False)
    df = df.set_index("timestamp").sort_index()

# Pick the column to forecast (adjust if your column name differs)
# Common names: 'close', 'price', 'value'
col_candidates = ["close", "price", "value", "y"]
col = next((c for c in col_candidates if c in df.columns), None)
assert col is not None, f"None of {col_candidates} found in CSV columns: {list(df.columns)}"

# Ensure regular sampling (optional, but helpful)
df = df[~df.index.duplicated(keep="last")].sort_index()
df = df.asfreq(FREQ)  # forward-fill gaps at 15T cadence; remove if you prefer strict filtering
df[col] = df[col].astype(float).interpolate("time").bfill().ffill()

# Final univariate series
y = df[col].copy()
y.name = "y"
print(y.index[:3], "…", y.index[-3:])
print("Length:", len(y), "| Freq:", pd.infer_freq(y.index))

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
               '2018-01-01 02:00:00'],
              dtype='datetime64[ns]', name='timestamp', freq='h') … DatetimeIndex(['2018-04-15 01:00:00', '2018-04-15 02:00:00',
               '2018-04-15 03:00:00'],
              dtype='datetime64[ns]', name='timestamp', freq='h')
Length: 2500 | Freq: h


/var/folders/pq/224wz8116dq05pr87bp6rkb80000gn/T/ipykernel_65911/4238514627.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.asfreq(FREQ)  # forward-fill gaps at 15T cadence; remove if you prefer strict filtering


In [11]:
import importlib, types

import moirai_no_gluonts  # this file should be next to the notebook
importlib.reload(moirai_no_gluonts)

from moirai_no_gluonts import forecast_no_lightning

In [16]:
import pkgutil, importlib, inspect, uni2ts

hits = []

def safe_import(name):
    try:
        return importlib.import_module(name)
    except Exception:
        return None

mods = [m.name for m in pkgutil.walk_packages(uni2ts.__path__, prefix=uni2ts.__name__ + ".")]
print(f"Scanning {len(mods)} modules under uni2ts…")

for mod_name in mods:
    m = safe_import(mod_name)
    if m is None:
        continue
    for name, obj in inspect.getmembers(m, inspect.isclass):
        has_generate = callable(getattr(obj, "generate", None))
        has_forward  = callable(getattr(obj, "forward", None)) or callable(obj)
        if not (has_generate or has_forward):
            continue
        qual = f"{obj.__module__}:{obj.__name__}"
        lname = obj.__name__.lower()
        score = 0
        if "moirai"   in lname: score += 3
        if "forecast" in lname or "module" in lname or "model" in lname: score += 1
        hits.append((score, qual, has_generate, has_forward))

hits.sort(key=lambda x: (-x[0], x[1]))
for s, qual, g, f in hits[:30]:
    print(f"[{s}] {qual} | generate={g} forward={f}")

MOIRAI_CLASS_CANDIDATE = hits[0][1] if hits else None
print("\nTop candidate:", MOIRAI_CLASS_CANDIDATE)

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [12]:
assert len(y) > CONTEXT_LENGTH + PREDICTION_LENGTH, "Series too short for chosen CONTEXT_LENGTH."

pred_vals, pred_idx = [], []
y_hist = y.iloc[:CONTEXT_LENGTH].copy()

for t in range(CONTEXT_LENGTH, len(y)):
    df_pred = forecast_no_lightning(
        y=y_hist,
        prediction_length=PREDICTION_LENGTH,
        context_length=CONTEXT_LENGTH,
        freq=FREQ,
        patch_size=PATCH_SIZE,
        rope_scale=ROPE_SCALE,
        repo_or_dir=os.environ.get("MOIRAI_REPO_OR_DIR", "."),
    )
    # last step point forecast
    pred_vals.append(float(df_pred["mean"].iloc[-1]))
    pred_idx.append(df_pred.index[-1])

    # teacher forcing: append true next step
    y_hist = pd.concat([y_hist, y.iloc[t:t+1]])

pred_rolling = pd.Series(pred_vals, index=pd.DatetimeIndex(pred_idx, name="timestamp"), name="y_pred")
print(f"Produced {len(pred_rolling)} one-step predictions.")

RuntimeError: Failed to import MOIRAI_CLASS='my_moirai_pkg.core:MoiraiModule'. Set MOIRAI_CLASS to 'pkg.module:ClassName'.

In [ ]:
# Align truth at the same timestamps as predictions
y_true_aligned = y.reindex(pred_rolling.index)

pred_df = pd.DataFrame({
    "timestamp": pred_rolling.index,
    "y_true":    y_true_aligned.values.astype(float),
    "y_pred":    pred_rolling.values.astype(float),
}).reset_index(drop=True)

pred_df.to_csv(OUT_PRED_CSV, index=False)
pred_df.head(5)